In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

Scrape data from Wikipedia's "significant earthquakes" lists over the past 120 years
- These lists contain earthquakes that are all:
    - Magnitude 6 or above (1900-2000), "unless they are notable for some other reason".
    - Magnitude 7 or above (2001-2020) or which caused fatalities.
    
In order to make the data more usable for purposes here, I will select all earthquakes from 1981 - 2020 that are magnitude 7 or above. This way, I can analyze all earthquakes that can be classified as "major" or "great" over the past 40 years.
    

In [434]:
url1 = "https://en.wikipedia.org/wiki/Lists_of_20th-century_earthquakes"
url2 = "https://en.wikipedia.org/wiki/List_of_earthquakes_2001%E2%80%932010"
url3 = "https://en.wikipedia.org/wiki/List_of_earthquakes_2011%E2%80%932020"

In [435]:
earthquakes_1 = pd.read_html(url1)
earthquakes_2 = pd.read_html(url2)
earthquakes_3 = pd.read_html(url3)


In [436]:
# Select only the tables from these lists that have 9 columns, since those are the ones we want:

new_earthquakes_1 = [table for table in earthquakes_1 if table.shape[1] ==9]
new_earthquakes_2 = [table for table in earthquakes_2 if table.shape[1] ==9]
new_earthquakes_3 = [table for table in earthquakes_3 if table.shape[1] ==9]
for table in range(len(new_earthquakes_1)):
    print(new_earthquakes_1[table].shape)
#for table in range(len(new_earthquakes_2)):
#    print(new_earthquakes_2[table].shape)
#for table in range(len(new_earthquakes_3)):
#    print(new_earthquakes_3[table].shape)

# Now, select only the last 2 tables from new_earthquakes_1 in order to get rid of data from years we don't want:
new_earthquakes_1 = [new_earthquakes_1[-2], new_earthquakes_1[-1]]

(26, 9)
(25, 9)
(29, 9)
(35, 9)
(36, 9)
(35, 9)
(42, 9)
(42, 9)
(40, 9)
(63, 9)


In [437]:
earthquakes_1_df = pd.concat(new_earthquakes_1)
earthquakes_2_df = pd.concat(new_earthquakes_2)
earthquakes_3_df = pd.concat(new_earthquakes_3)

In [438]:
earthquakes_1_df.head()

,Date,Time,Place,Lat.,Long.,Deaths,Magnitude,Comments,Sources
0,"January 19, 1981",15:11,"Irian Jaya, Indonesia see 1981 Irian Jaya eart...",−4.576,139.232,305.0,6.7,Ms,[86]
1,"January 23, 1981",21:13,"Sichuan, China see 1981 Dawu earthquake",30.93,101.10,150.0,6.8,ML,USGS
2,"October 25, 1981",03:22 UTC,"Playa Azul, Mexico see 1981 Playa Azul earthquake",18.05,-102.08,3.0,7.3,Ms,USGS
3,"December 13, 1982",09:12,"Dhamar, North Yemen (now Yemen) see 1982 North...",14.70,44.38,2800.0,6.2,Mw,USGS
4,"May 2, 1983",23:42,"Coalinga, California, United States see 1983 C...",36.23,−120.32,NaN,6.5,"Mw (Ellsworth, 1990)",USGS


In [439]:
earthquakes_2_df.head()

,Date,Time (UTC),Place,Lat.,Long.,Fatalities,Magnitude,Comments,Sources,Source
0,"January 1, 2001",06:57,"Mindanao, Philippines",6.898,126.579,0,7.5,Mw (HRV).,NaN,NaN
1,"January 9, 2001",16:49,Vanuatu,-14.928,167.170,0,7.1,Mw (USGS).,NaN,NaN
2,"January 10, 2001",16:02,"Kodiak Island region, Alaska, United States",57.078,-153.211,0,7.0,Mw (HRV).,NaN,NaN
3,"January 13, 2001",17:33,Offshore El Salvador see January 2001 El Salva...,13.049,-88.660,944,7.7,Mw (HRV).,NaN,NaN
4,"January 26, 2001",03:16,"Gujarat, India see 2001 Gujarat earthquake",23.419,70.232,20085,7.7,Mw (HRV).,NaN,NaN


In [440]:
earthquakes_3_df.head()

,Date,Time (UTC),Place,Lat.,Long.,Fatalities,Magnitude,Comments,Source
0,"January 1, 2011",09:56,"Santiago del Estero, Argentina",-26.794,-63.079,0,7.0,Mw (USGS) Centred 160 km NE of Santiago del Es...,[1]
1,"January 2, 2011",20:20,"Araucania, Chile see 2011 Temuco earthquake",-38.354,-73.275,0,7.1,"Mw (USGS) Centred 70 km NW of Temuco, Araucani...",[2]
2,"January 13, 2011",16:16,"Loyalty Islands, New Caledonia",-20.622,168.459,0,7.0,"Mw (USGS) Centred 120 km NNE of Tadine, Loyalt...",[3]
3,"January 18, 2011",20:23,Southwestern Pakistan see 2011 Dalbandin earth...,28.732,63.928,3,7.2,"Mw (USGS) Centred 50 km WSW of Dalbandin, Paki...",[4]
4,"February 4, 2011",13:53,Myanmar–India border region,24.618,94.680,1,6.2,"Mw (USGS) Centred 62.5 km east of Wangjing, In...",[5]


Now, select only the "Date", "Time (UTC)", "Lat.", "Long.", "Fatalities" (or "Deaths"), and "Magnitude" columns

In [442]:
earthquakes_1_df['Deaths'] = earthquakes_1_df['Deaths']
earthquakes_1df = earthquakes_1_df[['Date','Time','Lat.','Long.','Deaths','Magnitude']]
earthquakes_2df = earthquakes_2_df[['Date','Time (UTC)','Lat.','Long.','Fatalities','Magnitude']]
earthquakes_3df = earthquakes_3_df[['Date','Time (UTC)','Lat.','Long.','Fatalities','Magnitude']]

Rename earthquakes_1df "Deaths" column to "Fatalities" in order to be more consistent with column names of earthquakes_2df and earthquakes_3df

In [443]:
earthquakes_1df.columns = ['Date','Time (UTC)','Lat.','Long.','Fatalities','Magnitude']


We can now merge the three datasets together to create one big earthquake dataframe:

In [469]:
all_earthquakes = pd.concat([earthquakes_1df, earthquakes_2df, earthquakes_3df])
all_earthquakes.reset_index(inplace=True)

In [470]:
# Drop missing values (about 27 rows)

all_earthquakes = all_earthquakes.dropna()
all_earthquakes = all_earthquakes.drop(columns=['index'])
all_earthquakes.reset_index(inplace=True)
all_earthquakes.head()

,index,Date,Time (UTC),Lat.,Long.,Fatalities,Magnitude
0,0,"January 19, 1981",15:11,−4.576,139.232,305,6.7
1,1,"January 23, 1981",21:13,30.93,101.10,150,6.8
2,2,"October 25, 1981",03:22 UTC,18.05,-102.08,3,7.3
3,3,"December 13, 1982",09:12,14.70,44.38,2800,6.2
4,6,"October 28, 1983",14:06,44.09,−113.8,2,7


Create a "DateTime" column that combines "Date" and "Time (UTC)". Then create a "Month" column and an "Hour" column.

Next, clean up the fatalities column and include only deaths.

Lastly, select only the earthquakes where Magnitude is at least 7.

In [486]:
time_col = all_earthquakes['Time (UTC)'].apply(lambda x: re.findall('\d{2}\W\d{2}', str(x)))
time_col = time_col.apply(lambda x: x[0])
time_col = time_col.apply(lambda x: re.sub('\.', ':', str(x)))

all_earthquakes['Time (UTC)'] = time_col

all_earthquakes['DateTime'] = pd.to_datetime(all_earthquakes['Date'] + ' ' + all_earthquakes['Time (UTC)'])

all_earthquakes['Month'] = all_earthquakes['DateTime'].apply(lambda x: x.month)
all_earthquakes['Hour'] = all_earthquakes['DateTime'].apply(lambda x: x.hour)

In [487]:
all_earthquakes.head()

,index,Date,Time (UTC),Lat.,Long.,Fatalities,Magnitude,DateTime,Month,Hour
0,0,"January 19, 1981",15:11,−4.576,139.232,305,6.7,1981-01-19 15:11:00,1,15
1,1,"January 23, 1981",21:13,30.93,101.10,150,6.8,1981-01-23 21:13:00,1,21
2,2,"October 25, 1981",03:22,18.05,-102.08,3,7.3,1981-10-25 03:22:00,10,3
3,3,"December 13, 1982",09:12,14.70,44.38,2800,6.2,1982-12-13 09:12:00,12,9
4,6,"October 28, 1983",14:06,44.09,−113.8,2,7,1983-10-28 14:06:00,10,14


In [475]:
fatalities_column = all_earthquakes['Fatalities'].apply(lambda x: re.sub(',', '', str(x)))
fatalities_column = fatalities_column.apply(lambda x: re.sub('\.0','', str(x)))
all_inj_and_dead = fatalities_column.apply(lambda x: re.findall('\d+', str(x)))
all_deaths = all_inj_and_dead.apply(lambda x: x[0])
all_earthquakes['Fatalities'] = pd.to_numeric(all_deaths)

# Now, the "Fatalities" column only includes deaths (in numeric format).

In [397]:
magnitudes = all_earthquakes['Magnitude'].apply(lambda x: re.findall('\d+\.*\d*', str(x)))
all_earthquakes['Magnitude'] = magnitudes.apply(lambda x: float(x[0]))

In [398]:
new_df = all_earthquakes.loc[all_earthquakes['Magnitude']>=7]

In [399]:
new_df.head()

,index,Date,Time (UTC),Lat.,Long.,Fatalities,Magnitude,DateTime
2,2,"October 25, 1981",03:22,18.05,-102.08,3,7.3,1981-10-25 03:22:00
4,6,"October 28, 1983",14:06,44.09,−113.8,2,7.0,1983-10-28 14:06:00
7,13,"March 3, 1985",19:47,−33.24,−71.850,177,7.8,1985-03-03 19:47:00
8,14,"April 8, 1985",21:56,−34.04,−71.39,2,7.5,1985-04-08 21:56:00
9,15,"September 19, 1985",13:17,18.44,−102.36,10000,8.0,1985-09-19 13:17:00


Now, 